### Import Libraries

In [ ]:
from psaw import PushshiftAPI
#from pmaw import PushshiftAPI
from pathlib import Path 
# from datetime import datetime, timezone, timedelta  ## incase timezone modifcations are needed
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import os
import glob
# import dask.dataframe as dd
import math

PAR_LENGTH = 150 # in number of words
FILTER_REQ = 0.7 # evaluation value for number of paragraphs

### API and Search Keyword Setup

In [ ]:
files = glob.glob(r"C:\Users\MoRevolution\Desktop\College\P&R\Research_LowSES\Data Collection\Reddit Search Keywords/*.txt", 
                   recursive = True)

filenames = []
for i in range(len(files)): 
    filenames.append(files[i][files[i].rfind("\\")+1:-4])
    

In [ ]:
from psaw import PushshiftAPI
api = PushshiftAPI()

In [ ]:
keyword_list = []
master_keyword_list = []
filename_counter = 0

for f in files: 
    with open(f, 'r') as file:
        keyword_list = []
        for line in file:
            keyword_list.append(line.strip('\n')) 
            
    keyword_list = [i for i in keyword_list if i]
    master_keyword_list.append(keyword_list)

# print(master_keyword_list)

### Version 1 (without self text filter)

In [ ]:
subreddit_list = ["college", "AskReddit", "science", "psychology", "socialwork", "personalfinance", "financialaid"]
limit = 1000
num_workers = 16*2
columnNames = ['author', 'selftext', 'url', 'permalink', 'subreddit', 'title', 'keyword', 'keyword_category', 'Number_of_Paragraphs']
keyword_id = ["MAN", "MDF", "SVO"]

count = 0

df_final = pd.DataFrame()
for keywords in master_keyword_list: 
    for subreddit in subreddit_list: 
        for keyword in keywords: 
            subs = api.search_submissions(
                subreddit= subreddit,
                q= keyword,
                filter = ['author', 'selftext', 'url', 'permalink', 'subreddit', 'title'],
                limit = limit, 
                num_workers = num_workers
                # metadata = "false",   
                # max_results_per_request= 500
            )
            
            df = pd.DataFrame([rows.d_ for rows in subs])
            # df["keyword_category"] = keyword_id[count]
            df["keyword"] = keyword
            try: 
                df["Number_of_Paragraphs"] = df['selftext'].apply(lambda n: len(n.split())/PAR_LENGTH)
            except: 
                continue

            df = df[df["Number_of_Paragraphs"] >= FILTER_REQ]
            df_final = pd.concat([df_final, df])
    df_final.to_csv("Data Set"/keyword_id[count]+".csv", index= False)
    count += 1

In [ ]:
df_final.columns
df_final.size

Working Trial, only to categorize data frames into separate directories based on keywords used 

In [ ]:
# Suggestions
    ## write a script that tries out multiple word combinations for all of the keywords already provided 
    ## consider using cudf instead of pandas for faster dataframe processing
    ## test this code using dask for CPU parallel computing
subreddit_list = ["college", "AskReddit", "science", "psychology", "socialwork", "personalfinance", "financialaid"]
limit = 1000
num_workers = 16*2

for keyword_n in master_keyword_list:
    count = 0 
    for keywords in keyword_list: 
        for subreddit_n in subreddit_list:   
            subs = api.search_submissions(
                subreddit= subreddit_n,
                q= keywords,
                filter = [ 'author', 'selftext', 'url', 'permalink', 'subreddit', 'title'],
                # metadata = "false",   
                # max_results_per_request= 500
                limit = limit, 
                num_workers = num_workers
                )
            
            # Set display
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_colwidth', None)

            df = pd.DataFrame([row for row in subs])
            ddf = dd.from_pandas(df, npartitions=6)

            try: 
                ddf["Number of Paragraphs"] = ddf['selftext'].apply(lambda n: len(n.split())/PAR_LENGTH)
            except:

                continue
                
            df_filtered = ddf[ddf["Number of Paragraphs"] >= FILTER_REQ]

            filepath = Path('C:/Users/MoRevolution/Desktop/College/Data Dump/Reddit_1/Submissions/'+ subreddit_n +'/'+filenames[filename_counter]+'/Keyword_'+ str(count)+'.csv')
            filepath.parent.mkdir(parents = True, exist_ok = True)
            df_filtered.to_csv(filepath,index=False, single_file=True)
            count += 1 
    filename_counter += 1 

### Version 2 (selftext filter enable)

In [ ]:
subreddit_list = ["college", "AskReddit", "science", "psychology", "socialwork", "personalfinance", "financialaid"]
limit = 1000
num_workers = 16*2
columnNames = ['author', 'selftext', 'url', 'permalink', 'subreddit', 'title', 'keyword', 'keyword_category', 'Number_of_Paragraphs']
keyword_id = ["MAN", "MDF", "SVO"]

count = 0

df_final = pd.DataFrame()

for keywords in master_keyword_list: 
    for subreddit in subreddit_list: 
        for keyword in keywords: 
            subs = api.search_submissions(
                subreddit= subreddit,
                q= keyword,
                filter = ['author', 'selftext', 'url', 'permalink', 'subreddit', 'title'],
                # selftext = add  or to make limit categories and use OR for more subcategories
                limit = limit, 
                num_workers = num_workers
                # metadata = "false",   
                # max_results_per_request= 500
            )
            
            df = pd.DataFrame([rows.d_ for rows in subs])
            # df["keyword_category"] = keyword_id[count]
            df["keyword"] = keyword
            
            try: 
                df["Number_of_Paragraphs"] = df['selftext'].apply(lambda n: len(n.split())/PAR_LENGTH)
            except: 
                continue

            df = df[df["Number_of_Paragraphs"] >= FILTER_REQ]
            df_final = pd.concat([df_final, df])
    df_final.to_csv("Data Set"/keyword_id[count]+".csv", index= False)
    count += 1

### Data Cleaning
- Fix problem with apostrophes being replaced by three different symbols

In [ ]:
ddf = dd.read_csv('Data Set/Man.csv')
ddf.head()

In [ ]:
ddf['selftext'] = ddf['selftext'].replace('â€™','\'')

In [ ]:
ddf.to_csv("Data Set/Filtered and SQL/Man.csv",index = False, single_file = True)